In [4]:
import pandas as pd
import numpy as np

DIR='..\\..\\..\\..\\chunk\\hakaton\\'

In [2]:
df = pd.read_csv(f'{DIR}statistics-07-20.csv', sep=';',parse_dates=[9])
df.tail(2)

,index_train,length,car_number,destination_esr,adm,danger,gruz,loaded,operation_car,operation_date,...,operation_st_id,operation_train,receiver,rodvag,rod_train,sender,ssp_station_esr,ssp_station_id,tare_weight,weight_brutto
4189913,NaN,NaN,62827514,968302.0,NaN,NaN,NaN,1.0,80.0,2020-07-16 02:37:00,...,2.000039e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4189914,NaN,1.0,62827514,968302.0,20.0,NaN,161128.0,NaN,21.0,2020-07-16 04:28:00,...,2.000039e+09,NaN,98098048.0,60.0,NaN,0.0,NaN,NaN,249.0,NaN


In [3]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4189915 entries, 0 to 4189914
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   index_train       1967722 non-null  float64       
 1   length            2474781 non-null  float64       
 2   car_number        4189915 non-null  int64         
 3   destination_esr   4128494 non-null  float64       
 4   adm               1104718 non-null  float64       
 5   danger            111986 non-null   float64       
 6   gruz              2410454 non-null  float64       
 7   loaded            2543916 non-null  float64       
 8   operation_car     3648634 non-null  float64       
 9   operation_date    4189915 non-null  datetime64[ns]
 10  operation_st_esr  4189317 non-null  float64       
 11  operation_st_id   4189317 non-null  float64       
 12  operation_train   1967721 non-null  float64       
 13  receiver          2425592 non-null  float6

In [5]:
#удалить строки с недействительной операцией и отсутствующим номером станции
df.dropna(subset =['operation_car','operation_st_esr'],inplace=True)
df['operation_car']=df['operation_car'].astype(np.int32)
df['operation_st_esr']=df['operation_st_esr'].astype(np.int32)


In [6]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3648085 entries, 0 to 4189914
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   index_train       1426263 non-null  float64       
 1   length            1933236 non-null  float64       
 2   car_number        3648085 non-null  int64         
 3   destination_esr   3589830 non-null  float64       
 4   adm               1104509 non-null  float64       
 5   danger            111986 non-null   float64       
 6   gruz              1876183 non-null  float64       
 7   loaded            2543576 non-null  float64       
 8   operation_car     3648085 non-null  int32         
 9   operation_date    3648085 non-null  datetime64[ns]
 10  operation_st_esr  3648085 non-null  int32         
 11  operation_st_id   3648085 non-null  float64       
 12  operation_train   1426262 non-null  float64       
 13  receiver          1884047 non-null  float6

index_train - Индекс поезда  
length - Длина вагона  
car_number - Номер вагона  
destination_esr - ЕСР станции назначения вагона   
adm - Код АСОУП администрации-собственника вагона  
danger - Признак опасного груза в вагоне. 1 - в вагоне опасный груз,  null - иначе  
gruz - Код груза  
loaded - Признак груженого вагона. 1 -  груженый, 2- порожний  
operation_car - Код операции ВМ АСОУП  
operation_date - Дата операции  
operation_st_esr - ЕСР станции операции  
operation_st_id - ID станции операции  
operation_train - Код операции ПМ АСОУП  
receiver - ОКПО грузополучателя  
rodvag - Род вагона АСОУП (учетного)  
rod_train - Род поезда АСОУП  
sender - ОКПО грузоотправителя  
ssp_station_esr - ЕСР станция ССП  
ssp_station_id - id станции ССП  
tare_weight - Вес тары вагона (ц)  
weight_brutto - weight_brutto  

In [125]:
# формирование новой базы, содержащей номер станции, нормер вагона, 
# начало и конец цикла обмена грузом между станцией и вагоном, 
# длительность погрузки и/или разгрузки и общую длительность

#рабочие множества для определения категории операций
oper1x=set(range(10,20))
oper2x=set(range(20,30))
oper1x2x=set(range(10,30))
oper34=set([3,4])
oper1x2x=oper1x2x.union(oper34)
print(oper1x,oper2x,oper1x2x)


#для каждой станции и вагона проанализировать все события и выделить из них законченный цикл между 3 и 4.as_integer_ratio
#цикл обязательно должен содержать события 3,4, начало и конец
# 1х,2х погрузка и/или разгрузка
def operaions_chain(operation_st_esr,car_number,dfout=None):

    # print(operation_st_esr,car_number)
    # список операций с этим вагоном на этой станции, упорядоченный по времени
    df_tmp_ = df[(df.car_number == car_number)&(df.operation_st_esr==operation_st_esr)].sort_values(by='operation_date')
    # print(df_tmp.shape)
    # if df_tmp.shape[0]<2:return None 
    
    if set(df_tmp_['operation_car'].values).intersection(oper1x2x)>set([3,4]):
        #необходимый минимальный набор операций есть, продолжаем анализ
        pass
    else: 
        # print(f'не хватает 3 или4; пропущено operation_st_esr,car_number={operation_st_esr},{car_number}')
        return dfout

    lidf_tmp=[] #список  для разбиения на несколько подфреймов, если потребуется разбиение
    # for ind,val in enumerate(df_tmp['operation_car']):
    while df_tmp_.shape[0]>=4:
        col_op=df_tmp_['operation_car'] #столбец всех операций для данной станции/данного вагона
        #тогда возможно встретить несколько последовательностей 3-4,3-4...
        #проверить это и если да, то поделить найденный фрейм на куски
        isDivreq=False #надо делить или не надо
        for i in range(len(col_op)-1,2,-1):#проверять от конца к началу, чтобы захватить последний маркер 4 в случае дублирования
            if set(col_op.iloc[:i].values).intersection(set(col_op.iloc[i:].values))>=oper34:
                #если поделилось на два куска, в каждом из которых есть и 3 и 4
                if col_op.iloc[i]!=col_op.iloc[i-1]:
                    isDivreq=True
                    #если на границе тестовых сетов перещелкнуло с 4 на 3
                    #разделить исходный фрейм на две части для отдельной обработки
                    lidf_tmp.append(df_tmp_.iloc[i:])
                    df_tmp_=df_tmp_.iloc[:i] #продолжить с остальной частью подфрейма
                    break
        if not isDivreq: break #больше нет признаков некольких последовательностей
    
    lidf_tmp.append(df_tmp_) #добавить имеющийся кусок
          
    operaions_chain = pd.DataFrame() #набираем сюда итоговые строки для новой базы

    for df_tmp in lidf_tmp:
        col_op=df_tmp['operation_car'] #столбец всех операций для данной станции/данного вагона
        if not ((col_op.iloc[0]==3) and (col_op.iloc[-1]==4)): 
            #нарушен порядок. должно быть 3-4, пропустить эти данные
            continue

        new_row = {} #для формирования очередной выходной строки новой базы
        

        isFind=False  #нашлось ли в подфрейме операции погрузки/разгрузки
        if len(set(col_op.values).intersection(oper1x))>0 and len(set(col_op.values).intersection(oper2x))>0:
            #если фрейм включает и погрузку, и разгрузку, то вычислить обе длительности, 
            #искать Middle границу между погрузкой/разгрузкой
            # причем раздвоить фрейм по первому вхождению операции 2х, следующему после последнего 1х ,
            # или наоборот- по первому 1х после последнего 2х - в зависимости от порядка разгрузка/погрузка
            for i in range(1,len(col_op)-1):#проверять от начала к концу (без начала3 и конца4), чтобы захватить последний маркер 1х перед первым 2х или наоборот
                if len(set(col_op.iloc[:i].values).intersection(oper1x))>0 and len(set(col_op.iloc[i:].values).intersection(oper2x))>0:
                    #сначала идет погрузка
                    # значит есть длительность погрузки=первоевремя2х-первоевремя3;
                    # и значит есть длительность разгрузки=последнеевремя4-первоевремя2х;
                    new_row['1Xduration'] = df_tmp['operation_date'].iloc[i]-df_tmp['operation_date'].iloc[0]
                    new_row['2Xduration'] = df_tmp['operation_date'].iloc[-1]-df_tmp['operation_date'].iloc[i]
                    pass
                elif len(set(col_op.iloc[:i].values).intersection(oper2x))>0 and len(set(col_op.iloc[i:].values).intersection(oper1x))>0:
                    #сначала идет разгрузка
                    new_row['2Xduration'] = df_tmp['operation_date'].iloc[i]-df_tmp['operation_date'].iloc[0]
                    new_row['1Xduration'] = df_tmp['operation_date'].iloc[-1]-df_tmp['operation_date'].iloc[i]
                    pass
                else:
                    pass
                    continue #искать дальше
                new_row['startM_date']=df_tmp['operation_date'].iloc[i] #начало второй операции - середина процесса - по которой определяется длительность и погрузки и разгрузки
                new_row['startM_oper']=col_op.iloc[i] #сама эта серединная Middle операция
                #поделили подфрейм надвое, проверить не осталось ли во второй части и 2х, и 1х - что будет означать ошибку
                if len(set(col_op.iloc[i:].values).intersection(oper1x))>0 and len(set(col_op.iloc[i:].values).intersection(oper2x))>0:
                    print(f'ошибка при обработке operation_st_esr,car_number={operation_st_esr},{car_number}')
                    return dfout
                    display(df_tmp[['car_number',	'operation_st_esr',	'operation_date',	'operation_car']])
                    raise Exception('структура базы не оправдала ожидания')
                isFind=True
                break #деление закончено
            if not isFind: 
                display(df_tmp)
                raise Exception('что-то пошло не так')

        elif len(set(col_op.values).intersection(oper1x))>0: #фрейм включает только погрузку
            pass
            new_row['1Xduration'] = df_tmp['operation_date'].iloc[-1]-df_tmp['operation_date'].iloc[0]
            new_row['2Xduration']=pd.Timedelta(0, "S")
            isFind=True
        elif len(set(col_op.values).intersection(oper2x))>0: #фрейм включает только разгрузку
            pass
            new_row['2Xduration'] = df_tmp['operation_date'].iloc[-1]-df_tmp['operation_date'].iloc[0]
            new_row['1Xduration']=pd.Timedelta(0, "S")
            isFind=True
        else:
            pass #нет ни погрузки, ни разгрузки, - выкинуть этот кусок вообще
        
        if not isFind:
            continue

        new_row['end4_date']=df_tmp['operation_date'].iloc[-1]
        new_row['start3_date']=df_tmp['operation_date'].iloc[0]
        new_row['car_number']=car_number
        new_row['operation_st_esr']=operation_st_esr

        operaions_chain = operaions_chain.append(new_row, ignore_index=True)

    if operaions_chain.shape[0]>0:
        # приведем продолжительность операции к формату часы
        
        operaions_chain['1Xduration'] = (operaions_chain['1Xduration']/np.timedelta64(1, 'h')).astype(float)
        operaions_chain['2Xduration'] = (operaions_chain['2Xduration']/np.timedelta64(1, 'h')).astype(float)
        
        operaions_chain['duration'] = operaions_chain['1Xduration']+operaions_chain['2Xduration']
        operaions_chain=operaions_chain.reindex(columns=['operation_st_esr','car_number','start3_date','end4_date','1Xduration','2Xduration','duration'])

        if dfout is None:
            return operaions_chain
        dfout=dfout.append(operaions_chain, ignore_index=True)
        print(f'\r operation_st_esr,car_number={operation_st_esr},{car_number}',end="                   ...")
        return dfout
    else:
        # print(f'не хватает погр/разгр; пропущено operation_st_esr,car_number={operation_st_esr},{car_number}')
        return dfout
    

{10, 11, 12, 13, 14, 15, 16, 17, 18, 19} {20, 21, 22, 23, 24, 25, 26, 27, 28, 29} {3, 4, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}


In [57]:
#сколько всего записей будет обработано в operaions_chain
df_g=df.groupby(['operation_st_esr','car_number']).agg(
operation_date= pd.NamedAgg(column = 'operation_date', aggfunc = 'min'))
df_g.reset_index(inplace=True)
df_g.shape

(980492, 3)

In [ ]:
st_vag_op_dur = pd.DataFrame() #пополняемая база итоговой таблицы
for row in df_g.values:
    operation_st_esr,car_number=row[0],row[1] 
    st_vag_op_dur = operaions_chain(operation_st_esr,car_number, dfout=st_vag_op_dur)
print('finish ',st_vag_op_dur.shape)    

In [128]:
#10 часов - общее время работы скрипта формирования новой таблицы
VERSION='0'
st_vag_op_dur.to_csv(f'всестанции-время_{VERSION}.csv')